# Toma de decisiones

- Optimización estocástica
- Profesor: Fernando Elizalde Ramírez
- Juan Pablo Echeagaray González
- A00830646
- 16 de agosto del 2022

In [12]:
import numpy as np

## Criterios de decisión

In [13]:
def print_result(result: tuple) -> None:
    return f'Take a_{result[0]} with value {result[1]}'

In [14]:
def expected_value(decision_matrix: np.array, prob: np.array) -> np.array:
    """Cálculo de valores esperados dada una matriz de riesgo y un vector de probabilidades

    Args:
        - decision_matrix (np.array): Matriz en la que las entradas M_ij representan la ganancia/beneficio de la alternativa i para el estado j
        - prob (np.array): Vector de probabilidades para cada uno de los estados

    Raises:
        - ValueError: El vector de probabilidades contiene entradas menores a 0
        - ValueError: La s ValueError('Probabilities do not add to one')
uma de las entradas del vector de probabilidades no suma 1

    Returns:
        - np.array: Vector conteniendo el valor esperado de cada acción
    """

    if np.any(prob < 0):
        raise ValueError('Probability vector contains negative values')

    elif not np.isclose(np.sum(prob),1):
        raise ValueError('Probabilities do not add to one')

    expected = np.matmul(decision_matrix, prob)

    return expected

In [15]:
def wald_decision(retribution_matrix: np.array, mode: str) -> tuple:
    """Aplicación del criterio de Wald (minimax, maximin)

    Args:
        - retribution_matrix (np.array): Matriz de retribuciones
        - mode (str): 'min', 'max'

    Raises:
        - ValueError: El modo debe de ser minimizar o maximizar

    Returns:
        - tuple: Acción a tomar, Valor esperado de la acción tomada
    """    
    action = None

    # If the retribution matrix represents profits apply maximin
    if mode == 'max':
        vals = np.min(retribution_matrix, axis=1)
        action = np.argmax(vals)
    
    # If the retribution matrix represents loss, apply minimax
    elif mode == 'min':
        vals = np.max(retribution_matrix, axis=1)
        action = np.argmin(vals)

    else:
        raise ValueError('Mode has to either be min or max')

    return action + 1, vals[action]

In [16]:
def optimistic(retribution_matrix: np.array, mode: str) -> tuple:
    """Aplicación del criterio optimista (maximax, minimin)

    Args:
        - retribution_matrix (np.array): Matriz de retribución
        - mode (str): 'min', 'max'

    Raises:
        - ValueError: El modo debe de ser minimizar o maximizar

    Returns:
        - tuple: Acción a tomar, Valor esperado de la acción tomada
    """    
    action = None

    if mode == 'max':
        vals = np.max(retribution_matrix, axis=1)
        action = np.argmax(vals)

    elif mode == 'min':
        vals = np.min(retribution_matrix, axis=1)
        action = np.argmin(vals)

    else:
        raise ValueError('Mode has to either be min or max')

    return action + 1, vals[action]

In [17]:
def savage_decision(retribution_matrix: np.array, mode: str, cost_of_opportunity:bool = False) -> tuple:
    """Aplicación del criterio de Savage

    Args:
        - retribution_matrix (np.array): Matriz de retribución
        - mode (str): Modo de operación, 'min', 'max'
        - cost_of_opportunity (bool, optional): Si se debe de regresar el valor del costo de oportunidad. Defaults to False.

    Raises:
        - ValueError: El modo de operación debe de ser 'min' o 'max'

    Returns:
        - tuple: Acción a tomar, Valor esperado de la acción a tomar, Coste de oportunidad
    """    
    action, value = None, None
    
    # If the retribution matrix represents profits
    if mode == 'max':
        loss_matrix = np.max(retribution_matrix, axis=0) - retribution_matrix
        action, value = wald_decision(loss_matrix, 'min')

    # If the retribution matrix represents loss
    elif mode == 'min':
        loss_matrix = retribution_matrix - np.min(retribution_matrix, axis=0)
        action, value = wald_decision(loss_matrix, 'min')

    else:
        raise ValueError('Mode has to either be min or max')
    
    # No need to add 1 to the action since its done inside wald_decision
    if cost_of_opportunity:
        print(loss_matrix)
        cost = np.min(np.sum(loss_matrix, axis=1))
        return action, value, cost

    return action, value

In [18]:
def laplace_decision(retribution_matrix: np.array, mode: str) -> tuple:
    """Aplicación del criterio de LaPlace

    Args:
        - retribution_matrix (np.array): Matriz de retribución
        - mode (str): Modo de operación, 'min', 'max'

    Raises:
        - ValueError: El modo de operación debe de ser 'min' o 'max'

    Returns:
        - tuple: Acción a tomar, Valor esperado de la acción a tomar
    """    
    action = None
    averages = np.average(retribution_matrix, axis=1)

    if mode == 'max':
        action = np.argmax(averages, axis=0)
    
    elif mode == 'min':
        action = np.argmin(averages, axis=0)
    
    else:
        raise ValueError('Mode has to either be min or max')

    return action + 1, averages[action]

## Problema 1

National Outdoors School (NOS) está preparando un sitio para acampar en el verano en el corazón de Alaska para enseñar técnicas de supervivencia en áreas salvajes. NOS estima que la asistencia puede caer dentro de una de cuatro categorías: $200, 250, 300, 350$ personas. El costo del campamento será mínimo cuando su tamaño satisfaga la demanda con exactitud. Las desviaciones por encima y por debajo de los niveles de demanda ideales incurren en costos adicionales por construir más capacidad que la necesaria o por perder oportunidades de ingresos cuando la demanda no se satisface. Si $a_1$ a $a_4$ representan los tamaños de los campamentos ($200, 250, 300, 350$ personas) y $s_1 \ldots s_4$ el nivel de asistencia, la siguiente tabla resume la matriz de costos (en miles de dólares) para la situación.

$$\begin{equation}
    \begin{bmatrix}
        5 & 10 & 18 & 25 \\
        8 & 7 & 12 & 23 \\
        21 & 18 & 12 & 21 \\
        30 & 22 & 19 & 15
    \end{bmatrix}
\end{equation}$$

In [19]:
ret = np.array([[5, 10, 18, 25],
                [8, 7, 12, 23],
                [21, 18, 12, 21],
                [30, 22, 19, 15]])

print(f'''Decisiones
Laplace = {print_result(laplace_decision(ret, 'min'))}
Wald = {print_result(wald_decision(ret, 'min'))}
Savage = {print_result(savage_decision(ret, 'min'))}''')

Decisiones
Laplace = Take a_2 with value 12.5
Wald = Take a_3 with value 21
Savage = Take a_2 with value 8


## Problema 2

Una empresa dedicada a la fabricación de calzado tiene que analizar entre diferentes estrategias de producción, aquella que le proporcione más ventas, y, en consecuencia, más beneficios. Los posibles productos son: botas, zapatos y sandalias.

La decisión la debe tomar en función de las predicciones del tiempo que haga en los próximos meses, ya que esto determinará que se venda más un producto u otro. Los estados de la naturaleza previstos son tres: tiempo frío, normal y cálido.

En el momento de tomar la decisión el empresario no sabe con seguridad el estado de tiempo, pero consultando los estados climáticos de los últimos años llega a las siguientes estimaciones en forma de probabilidad: existe un 30% de probabilidad de que el tiempo sea frío, un 45% de que sea normal, y un 25% de que sea cálido.

Por otro lado, la experiencia en el sector le permite estimar los resultados esperados en cuanto a ventas, y esto le permite elaborar las siguientes predicciones o desenlaces:

- La fabricación de botas le daría unos beneficios (en euros) de 60.000, 15.000 y 2.500, si el tiempo es frío, normal o cálido respectivamente.
- La fabricación de zapatos le daría unos beneficios (en euros) de 5.000, 30.000 y 10.000, si el tiempo es frío, normal o cálido respectivamente.
- La fabricación de sandalias le daría unos beneficios (en euros) de -5.000, 7.500 y 50.000, si el tiempo es frío, normal o cálido respectivamente.

### Situación de riesgo

- Elaborar la matriz de decisión.
- Calcular los valores esperados de cada una de las estrategias.

In [20]:
ret_2 = np.array([[60_000, 15_000, 2_500],
                  [5_000, 30_000, 10_000],
                  [-5_000, 7_500, 50_000]])
prob = np.array([0.3, 0.45, 0.25])

print(ret_2)

print(f'VE(B, Z, S) = {expected_value(ret_2, prob)}')


[[60000 15000  2500]
 [ 5000 30000 10000]
 [-5000  7500 50000]]
VE(B, Z, S) = [25375. 17500. 14375.]


### Situación de incertidumbre

- Opción elegida con criterio pesimista
- Opción elegida con criterio optimista
- Opción elegida según el criterio de LaPlace
- Opción elegida con criterio de Savage

In [21]:
labels = {'a1': 'botas', 'a2': 'zapatos', 'a3': 'sandalias'}
action, _, cost = savage_decision(ret_2, 'max', True)

print(f'''Opciones elegidas:
Etiquetas: {labels}
Pesimista: {print_result(wald_decision(ret_2, 'max'))}
Optimista: {print_result(optimistic(ret_2, 'max'))}
LaPlace: {print_result(laplace_decision(ret_2, 'max'))}
Savage: Take a_{action} with opportunity loss of {cost}''')


[[    0 15000 47500]
 [55000     0 40000]
 [65000 22500     0]]
Opciones elegidas:
Etiquetas: {'a1': 'botas', 'a2': 'zapatos', 'a3': 'sandalias'}
Pesimista: Take a_2 with value 5000
Optimista: Take a_1 with value 60000
LaPlace: Take a_1 with value 25833.333333333332
Savage: Take a_1 with opportunity loss of 62500
